<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/Recent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import re
lab = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/Lab.csv')

In [53]:
df0 = pd.DataFrame(lab)

In [54]:
import pandas as pd
df0 = pd.DataFrame(lab)
df = pd.DataFrame(df0)
df = df[['PatientUID', 'ResultItemName', 'ResultValue', 'CWhen']]
df['ResultItemName'] = df['ResultItemName'].replace(['CREATININE, 32202'], 'Creatinine')
df = df[df['ResultItemName'].isin(['eGFR(CKD-EPI) age >=18', 'Creatinine'])]
pivot = df.pivot(index=['PatientUID', 'CWhen'], columns='ResultItemName', values='ResultValue').reset_index()
sort = pivot.sort_values('CWhen').groupby('PatientUID').tail(1)
sort[['Creatinine', 'eGFR(CKD-EPI) age >=18']] = sort.groupby('PatientUID')[['Creatinine', 'eGFR(CKD-EPI) age >=18']].apply(lambda x: x.ffill().bfill())
result = sort.rename(columns={'Creatinine': 'Recent_Creatinine', 'eGFR(CKD-EPI) age >=18': 'Recent_eGFR', 'CWhen':'Recent_Date'})

<ipython-input-54-97383c362ca0>:9: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  sort[['Creatinine', 'eGFR(CKD-EPI) age >=18']] = sort.groupby('PatientUID')[['Creatinine', 'eGFR(CKD-EPI) age >=18']].apply(lambda x: x.ffill().bfill())


In [55]:
# Fill the NaN values
import numpy as np

# Fill specific NaN values
result['Recent_eGFR'] = result['Recent_eGFR'].mask(result['PatientUID'] == 21282, result['Recent_eGFR'].fillna(55.50))
result['Recent_eGFR'] = result['Recent_eGFR'].mask(result['PatientUID'] == 173101, result['Recent_eGFR'].fillna(72.85))

result['Recent_Creatinine'] = result['Recent_Creatinine'].mask(result['PatientUID'] == 792446, result['Recent_Creatinine'].fillna(1.74))

# Verify if there are still any null values
print(result.isnull().any())

ResultItemName
PatientUID           False
Recent_Date          False
Recent_Creatinine    False
Recent_eGFR          False
dtype: bool


In [59]:
merge_df = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/SummaryFile/WideTable/WideTable.xlsx')

In [60]:
merge_df.shape

(52619, 84)

In [64]:
data = pd.merge(result[['PatientUID', 'Recent_Creatinine', 'Recent_eGFR']], merge_df, on='PatientUID')

In [65]:
data

,PatientUID,Recent_Creatinine,Recent_eGFR,Unnamed: 0,PatientVisitUID,Slopes,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,...,L14:WBC,Sex,Age,NATNL,Location,PricipalProblemCode,PricipalProblemName,InPatient,OutPatient,Emergency
0,924852,2.63,15.80,27771,4376081,18.847273,0,0,0,0,...,6.67,หญิง,87.0,ไทย,อายุรกรรม 8 (OPD)-D07,44054006,Type 2 diabetes mellitus,0.0,1.0,0.0
1,989278,2.03,32.30,29564,4390501,4.319167,63,62,0,90,...,9.65,ชาย,70.0,ไทย,โรคมะเร็งต่อมน้ำเหลือง (คลินิก)-D720,55921005,Multiple myeloma,0.0,1.0,0.0
2,2103792,1.05,55.90,46554,4394805,0.882418,0,0,0,0,...,7.48,หญิง,65.0,ไทย,OPD AE-DV191,93880001,Primary malignant neoplasm of lung,0.0,0.0,1.0
3,953611,1.03,51.10,28634,4395019,6.312446,0,0,0,0,...,NaN,หญิง,81.0,ไทย,OPD AE-DV191,S9080,Abrasion of ankle and foot,0.0,0.0,1.0
4,488150,1.94,32.60,16402,4396053,2.848364,0,0,0,0,...,4.66,หญิง,36.0,ไทย,อายุรกรรม 8 (OPD)-D07,M321,Systemic lupus erythematosus with organ or sys...,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,43763,1.30,55.28,3086,5674644,-9.146778,0,0,0,0,...,NaN,ชาย,66.0,ไทย,หน่วยวางแผนครอบครัว (OPD)-D20,73211009,Diabetes mellitus,0.0,1.0,0.0
52615,43763,1.30,55.28,3087,6817059,-4.929870,0,0,0,0,...,NaN,ชาย,67.0,ไทย,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,NaN,NaN,0.0,1.0,0.0
52616,43763,1.30,55.28,3088,8004571,16.017287,0,0,0,0,...,NaN,ชาย,68.0,ไทย,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,NaN,NaN,0.0,1.0,0.0
52617,43763,1.30,55.28,3089,9214741,-24.861814,0,0,0,0,...,NaN,ชาย,70.0,ไทย,อายุรกรรม 8 (OPD)-D07,E11,Non-insulin-dependent diabetes mellitus,0.0,1.0,0.0


In [68]:
visit = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/Visit.csv')


<ipython-input-68-14de829f4d7b>:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  visit = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/Visit.csv')


In [69]:
visit.columns

Index(['PatientUID', 'PatientvisitUID', 'Sex', 'Age', 'ENTY', 'NATNL',
       'StartDTTM', 'EndDttm', 'Location', 'PricipalProblemCode',
       'PricipalProblemName'],
      dtype='object')

In [ ]:
# Step 1: Sort the dataframe by date to ensure the latest location comes last for each patient
visit['VisitDate'] = pd.to_datetime(visit['VisitDate'])
visit.sort_values(['PatientUID', 'VisitDate'], ascending=[True, False], inplace=True)

# Step 2: Get the latest location for each patient
latest_location = visit.groupby('PatientUID').first().reset_index()[['PatientUID', 'Location']]

# Step 3: Create a dictionary mapping patient UID to the latest location
latest_location_dict = dict(zip(latest_location['PatientUID'], latest_location['Location']))

# Step 4: Use the dictionary to fill in missing values in the 'LabLocation' column
merged_df['LatestLocation'] = merged_df.apply(lambda row: latest_location_dict[row['PatientUID']] if pd.isnull(row['Location']) else row['Location'], axis=1)
